In [1]:
import requests
import json
import pandas as pd

In [28]:
draft_database = pd.read_csv('NHL_Drafts.csv')

In [29]:
draft_database['Full_Name'] = draft_database['Player_Name'].apply(lambda x: x.split()[0]+ ' ' + x.split()[1])
draft_database['Position'] = draft_database['Player_Name'].apply(lambda x: x.split()[2])
del draft_database['Player_Name']

In [182]:
draft_database[(draft_database['Year'] == 2015) & (draft_database['Position'] == '(G)')]
# draft_database.drop_duplicates()

,Year,Round,Overall_Pick,Nationality,Full_Name,Position
21,2015,1,22,Russia,Ilya Samsonov,(G)
41,2015,2,42,Canada,Mackenzie Blackwood,(G)
69,2015,3,70,Sweden,Felix Sandstrom,(G)
74,2015,3,75,Czech Republic,Dan Vladar,(G)
75,2015,3,76,Canada,Adin Hill,(G)
76,2015,3,77,Canada,Samuel Montembeault,(G)
85,2015,3,86,United States,Mike Robinson,(G)
89,2015,3,90,Slovakia,Matej Tomek,(G)
92,2015,4,93,Canada,Callum Booth,(G)
110,2015,4,111,Czech Republic,Ales Stezka,(G)


In [ ]:
#Draft Picks By Round NOT GOALIE
# 1: 153
# 2: 155
# 3: 153
# 4: 153
# 5: 153
# 6: 153
# 7: 153

In [47]:
def return_a_player_stats(a_name):
    try:
        base_url = 'https://suggest.svc.nhl.com/svc/suggest/v1/minplayers/'
        first_name = a_name.split()[0]
        last_name = a_name.split()[1]
        num_to_return = '1'

        full_url = base_url + first_name + '%20' + last_name + '/' + num_to_return
        response = requests.get(full_url)
        suggestions = json.loads(response.content)['suggestions'][0]
        player_info = str.split(suggestions, "|")
        player_id = player_info[0]
        url = 'https://statsapi.web.nhl.com/api/v1/people/' + player_id + '/stats/?stats=yearByYear'
        response = requests.get(url)
        content = json.loads(response.content)['stats']
        splits = content[0]['splits']

        df_splits = (pd.json_normalize(splits, sep = "_" )
                .query('league_name == "National Hockey League"')
        )
        df_splits['Full_Name'] = a_name

        return df_splits
    except IndexError:
        return 5

In [50]:
return_a_player_stats('Rasmus Dahlin')

,season,sequenceNumber,stat_assists,stat_goals,stat_pim,stat_games,stat_penaltyMinutes,stat_points,team_name,team_link,...,stat_gameWinningGoals,stat_overTimeGoals,stat_shortHandedGoals,stat_shortHandedPoints,stat_shortHandedTimeOnIce,stat_blocked,stat_shifts,team_id,league_id,Full_Name
25,20182019,1,35,9,34,82,34,44,Buffalo Sabres,/api/v1/teams/7,...,4.0,0.0,0.0,0.0,12:59,66.0,1905.0,7.0,133.0,Rasmus Dahlin
26,20192020,1,36,4,38,59,38,40,Buffalo Sabres,/api/v1/teams/7,...,1.0,0.0,0.0,0.0,05:44,51.0,1217.0,7.0,133.0,Rasmus Dahlin
27,20202021,1,18,5,26,56,26,23,Buffalo Sabres,/api/v1/teams/7,...,0.0,0.0,0.0,0.0,04:40,45.0,1242.0,7.0,133.0,Rasmus Dahlin
28,20212022,1,40,13,68,80,68,53,Buffalo Sabres,/api/v1/teams/7,...,3.0,1.0,0.0,1.0,72:58,91.0,2317.0,7.0,133.0,Rasmus Dahlin
31,20222023,1,49,14,76,58,76,63,Buffalo Sabres,/api/v1/teams/7,...,2.0,0.0,0.0,0.0,84:43,100.0,1784.0,7.0,133.0,Rasmus Dahlin


In [65]:
def create_a_stats_db():
    list_of_names = list(draft_database['Full_Name'])
    mega_db = pd.DataFrame()

    for name in list_of_names:
        temp_db = return_a_player_stats(name)
        if isinstance(temp_db,int):
            pass
        mega_db = pd.concat([mega_db, temp_db], axis=0,ignore_index=True,sort=False) 

    return mega_db

In [128]:
return_a_player_stats('Juuso Parssinen')

,season,sequenceNumber,stat_assists,stat_goals,stat_pim,stat_games,stat_penaltyMinutes,stat_points,team_name,team_link,...,stat_gameWinningGoals,stat_overTimeGoals,stat_shortHandedGoals,stat_shortHandedPoints,stat_shortHandedTimeOnIce,stat_blocked,stat_shifts,team_id,league_id,Full_Name
16,20222023,1,18,5,15,42,15,23,Nashville Predators,/api/v1/teams/18,...,2.0,0.0,0.0,0.0,00:57,16.0,727.0,18.0,133.0,Juuso Parssinen


In [66]:
df = create_a_stats_db()

In [79]:
aggregated_df = pd.merge(draft_database,df,on='Full_Name')

In [80]:
# aggregated_df.to_csv('2015-2019_Draft_Players_all_seasons.csv',index=False)
aggregated_df

,Year,Round,Overall_Pick,Nationality,Full_Name,Position,season,sequenceNumber,stat_assists,stat_goals,...,stat_powerPlaySaves,stat_shortHandedSaves,stat_evenSaves,stat_shortHandedShots,stat_evenShots,stat_powerPlayShots,stat_gamesStarted,stat_powerPlaySavePercentage,stat_shortHandedSavePercentage,stat_evenStrengthSavePercentage
0,2015,1,1,Canada,Connor McDavid,(C),20152016,1,32.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,Canada,Connor McDavid,(C),20162017,1,70.0,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,Canada,Connor McDavid,(C),20172018,1,67.0,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,Canada,Connor McDavid,(C),20182019,1,75.0,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,Canada,Connor McDavid,(C),20192020,1,63.0,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,2019,6,184,Finland,Santeri Hatakka,(D),20212022,1,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1616,2019,7,201,Canada,Rafael Harvey-Pinard,(LW),20212022,1,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1617,2019,7,201,Canada,Rafael Harvey-Pinard,(LW),20222023,1,4.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1618,2019,7,203,Finland,Valtteri Puustinen,(RW),20212022,1,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
#Now I must trim to only include 3 seasons from Draft Date
aggregated_df['season_played'] = aggregated_df.apply(lambda x: int(x['season'][4:]) - int(x['Year']),axis=1)
#If season_played is greater than 3 we should ignore

In [90]:
aggregated_df = aggregated_df[(aggregated_df['season_played'] <= 3) & (aggregated_df['season_played'] >= 1)] 

In [91]:
goalies_only = aggregated_df[aggregated_df['Position'] == '(G)']

In [93]:
goalies_only.to_csv('Goalie_Stats.csv',index=False)

In [124]:
all_other_players = aggregated_df[aggregated_df['Position'] != '(G)']
all_other_players = all_other_players[all_other_players['Full_Name'] != 'Sebastian Aho']
all_other_players = all_other_players[all_other_players['Full_Name'] != 'Ryan O\'Reilly']
all_other_players.drop_duplicates()

,Year,Round,Overall_Pick,Nationality,Full_Name,Position,season,sequenceNumber,stat_assists,stat_goals,...,stat_shortHandedSaves,stat_evenSaves,stat_shortHandedShots,stat_evenShots,stat_powerPlayShots,stat_gamesStarted,stat_powerPlaySavePercentage,stat_shortHandedSavePercentage,stat_evenStrengthSavePercentage,season_played
0,2015,1,1,Canada,Connor McDavid,(C),20152016,1,32.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2015,1,1,Canada,Connor McDavid,(C),20162017,1,70.0,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,2015,1,1,Canada,Connor McDavid,(C),20172018,1,67.0,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
8,2015,1,2,United States,Jack Eichel,(C),20152016,1,32.0,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9,2015,1,2,United States,Jack Eichel,(C),20162017,1,33.0,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1608,2019,4,98,Finland,Matias Maccelli,(LW),20212022,1,5.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1611,2019,5,125,United States,Mark Kastelic,(C),20212022,1,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1615,2019,6,184,Finland,Santeri Hatakka,(D),20212022,1,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1616,2019,7,201,Canada,Rafael Harvey-Pinard,(LW),20212022,1,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


In [118]:
all_other_players.columns

Index(['Year', 'Round', 'Overall_Pick', 'Nationality', 'Full_Name', 'Position',
       'season', 'sequenceNumber', 'stat_assists', 'stat_goals', 'stat_pim',
       'stat_games', 'stat_penaltyMinutes', 'stat_points', 'team_name',
       'team_link', 'league_name', 'league_link', 'team_id', 'league_id',
       'stat_powerPlayGoals', 'stat_gameWinningGoals', 'stat_shortHandedGoals',
       'stat_plusMinus', 'stat_shots', 'stat_timeOnIce', 'stat_hits',
       'stat_powerPlayPoints', 'stat_powerPlayTimeOnIce', 'stat_evenTimeOnIce',
       'stat_faceOffPct', 'stat_shotPct', 'stat_overTimeGoals',
       'stat_shortHandedPoints', 'stat_shortHandedTimeOnIce', 'stat_blocked',
       'stat_shifts', 'stat_savePercentage', 'stat_goalAgainstAverage',
       'stat_shutouts', 'stat_ties', 'stat_wins', 'stat_losses',
       'stat_goalsAgainst', 'stat_shotsAgainst', 'stat_ot', 'stat_saves',
       'stat_powerPlaySaves', 'stat_shortHandedSaves', 'stat_evenSaves',
       'stat_shortHandedShots', 'stat_eve

In [119]:

a_list = ['stat_savePercentage', 'stat_goalAgainstAverage',
       'stat_shutouts', 'stat_ties', 'stat_wins', 'stat_losses',
       'stat_goalsAgainst', 'stat_shotsAgainst', 'stat_ot', 'stat_saves',
       'stat_powerPlaySaves', 'stat_shortHandedSaves', 'stat_evenSaves',
       'stat_shortHandedShots', 'stat_evenShots', 'stat_powerPlayShots',
       'stat_gamesStarted', 'stat_powerPlaySavePercentage',
       'stat_shortHandedSavePercentage', 'stat_evenStrengthSavePercentage']

for stat in a_list:
    del all_other_players[stat]

In [120]:
all_other_players.to_csv('2015-2019-3Years-SkaterStats.csv', index=False)

In [103]:
#Avalible Player Stats
# Index(['Year', 'Round', 'Overall_Pick', 'Nationality', 'Full_Name', 'Position',
#        'season', 'sequenceNumber', 'stat_assists', 'stat_goals', 'stat_pim',
#        'stat_games', 'stat_penaltyMinutes', 'stat_points', 'team_name',
#        'team_link', 'league_name', 'league_link', 'team_id', 'league_id',
#        'stat_powerPlayGoals', 'stat_gameWinningGoals', 'stat_shortHandedGoals',
#        'stat_plusMinus', 'stat_shots', 'stat_timeOnIce', 'stat_hits',
#        'stat_powerPlayPoints', 'stat_powerPlayTimeOnIce', 'stat_evenTimeOnIce',
#        'stat_faceOffPct', 'stat_shotPct', 'stat_overTimeGoals',
#        'stat_shortHandedPoints', 'stat_shortHandedTimeOnIce', 'stat_blocked',
#        'stat_shifts', 'season_played'],
#       dtype='object')

Index(['Year', 'Round', 'Overall_Pick', 'Nationality', 'Full_Name', 'Position',
       'season', 'sequenceNumber', 'stat_assists', 'stat_goals', 'stat_pim',
       'stat_games', 'stat_penaltyMinutes', 'stat_points', 'team_name',
       'team_link', 'league_name', 'league_link', 'team_id', 'league_id',
       'stat_powerPlayGoals', 'stat_gameWinningGoals', 'stat_shortHandedGoals',
       'stat_plusMinus', 'stat_shots', 'stat_timeOnIce', 'stat_hits',
       'stat_powerPlayPoints', 'stat_powerPlayTimeOnIce', 'stat_evenTimeOnIce',
       'stat_faceOffPct', 'stat_shotPct', 'stat_overTimeGoals',
       'stat_shortHandedPoints', 'stat_shortHandedTimeOnIce', 'stat_blocked',
       'stat_shifts', 'season_played'],
      dtype='object')

In [126]:
all_other_players.groupby(['Round'])['stat_points'].mean()

Round
1    20.443478
2     6.662162
3     7.300000
4     6.789474
5     6.090909
6     9.888889
7     6.600000
Name: stat_points, dtype: float64

In [175]:
all_other_players[all_other_players['Round']==1]

,Year,Round,Overall_Pick,Nationality,Full_Name,Position,season,sequenceNumber,stat_assists,stat_goals,...,stat_shortHandedSaves,stat_evenSaves,stat_shortHandedShots,stat_evenShots,stat_powerPlayShots,stat_gamesStarted,stat_powerPlaySavePercentage,stat_shortHandedSavePercentage,stat_evenStrengthSavePercentage,season_played
0,2015,1,1,Canada,Connor McDavid,(C),20152016,1,32.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2015,1,1,Canada,Connor McDavid,(C),20162017,1,70.0,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,2015,1,1,Canada,Connor McDavid,(C),20172018,1,67.0,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
8,2015,1,2,United States,Jack Eichel,(C),20152016,1,32.0,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9,2015,1,2,United States,Jack Eichel,(C),20162017,1,33.0,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1538,2019,1,25,Canada,Connor McMichael,(C),20202021,1,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1539,2019,1,25,Canada,Connor McMichael,(C),20212022,1,9.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1542,2019,1,27,Canada,Nolan Foote,(LW),20202021,1,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1543,2019,1,27,Canada,Nolan Foote,(LW),20212022,1,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
